In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from datetime import datetime
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DecimalType

In [74]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [38]:
warehouse_location = '/files'

In [39]:
spark = SparkSession \
    .builder \
    .appName("App de Spark para QQP") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .getOrCreate()

24/05/27 03:05:48 INFO SparkEnv: Registering MapOutputTracker
24/05/27 03:05:48 INFO SparkEnv: Registering BlockManagerMaster
24/05/27 03:05:48 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/05/27 03:05:48 INFO SparkEnv: Registering OutputCommitCoordinator


In [40]:
spark

In [44]:
!hdfs dfs -ls /user/QQP | head -n 6

Found 452 items
-rw-r--r--   2 root hadoop  180751333 2024-05-27 02:39 /user/QQP/01-2023_01.csv
-rw-r--r--   2 root hadoop  234811063 2024-05-27 02:37 /user/QQP/01-2023_02.csv
-rw-r--r--   2 root hadoop  178013160 2024-05-27 02:17 /user/QQP/01-2024_01.csv
-rw-r--r--   2 root hadoop  226158794 2024-05-27 02:17 /user/QQP/01-2024_02.csv
-rw-r--r--   2 root hadoop   16997440 2024-05-27 01:45 /user/QQP/012015.csv
-rw-r--r--   2 root hadoop   97734005 2024-05-27 02:28 /user/QQP/012017.csv
-rw-r--r--   2 root hadoop   80231751 2024-05-27 02:23 /user/QQP/012018.csv
-rw-r--r--   2 root hadoop   45064083 2024-05-27 02:42 /user/QQP/012019.csv
-rw-r--r--   2 root hadoop   30695790 2024-05-27 02:26 /user/QQP/012020.csv
-rw-r--r--   2 root hadoop  203295640 2024-05-27 02:38 /user/QQP/02-2023_01.csv
-rw-r--r--   2 root hadoop  180993906 2024-05-27 02:38 /user/QQP/02-2023_02.csv
-rw-r--r--   2 root hadoop  189448783 2024-05-27 02:18 /user/QQP/02-2024_01.csv
-rw-r--r--   2 root hadoop  198036958 2024-0

In [ ]:
%%time
!hdfs dfs -count /user/QQP

In [58]:
import subprocess

dir_in = "/user/QQP"
args = "hdfs dfs -ls "+dir_in+" | awk '{print $8}'"
proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

s_output, s_err = proc.communicate()
all_dart_dirs = s_output.split()

In [30]:
schema = StructType([
    StructField("PRODUCTO", StringType(), True),
    StructField("PRESENTACIÓN", StringType(), True),
    StructField("MARCA", StringType(), True),
    StructField("CATEGORÍA", StringType(), True),
    StructField("CATÁLOGO", StringType(), True),
    StructField("PRECIO", DecimalType(18, 2), True),
    StructField("FECHAREGISTRO", StringType(), True),
    StructField("CADENACOMERCIAL", StringType(), True),
    StructField("GIRO", StringType(), True),
    StructField("NOMBRECOMERCIAL", StringType(), True),
    StructField("DIRECCIÓN", StringType(), True),
    StructField("ESTADO", StringType(), True),
    StructField("MUNICIPIO", StringType(), True),
    StructField("LATITUD", DecimalType(18, 6), True),
    StructField("LONGITUD", DecimalType(18, 6), True)
])

In [64]:
final_df = spark.createDataFrame([], schema)

In [69]:
%%time
for csv_dir in all_dart_dirs:
    # print(csv_dir.decode('utf-8'))
    df = spark.read.csv(csv_dir.decode('utf-8'), sep=',', header=False, schema=schema)
    final_df = final_df.union(df)

In [70]:
%%time
final_df.count()

24/05/27 03:38:03 WARN DAGScheduler: Broadcasting large task binary with size 1457.3 KiB


CPU times: user 1.13 s, sys: 321 ms, total: 1.45 s
Wall time: 12min 4s


272944716

In [72]:
final_df.printSchema()

root
 |-- PRODUCTO: string (nullable = true)
 |-- PRESENTACIÓN: string (nullable = true)
 |-- MARCA: string (nullable = true)
 |-- CATEGORÍA: string (nullable = true)
 |-- CATÁLOGO: string (nullable = true)
 |-- PRECIO: decimal(18,2) (nullable = true)
 |-- FECHAREGISTRO: timestamp (nullable = true)
 |-- CADENACOMERCIAL: string (nullable = true)
 |-- GIRO: string (nullable = true)
 |-- NOMBRECOMERCIAL: string (nullable = true)
 |-- DIRECCIÓN: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- LATITUD: decimal(18,6) (nullable = true)
 |-- LONGITUD: decimal(18,6) (nullable = true)



In [76]:
# final_df.describe().show(5, False)

24/05/27 03:57:07 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
ERROR:root:KeyboardInterrupt while sending command.             (20 + 6) / 1810]
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [77]:
final_df.createOrReplaceTempView("QQP")

In [79]:
%%time
df_test = spark.sql(""" Select * from QQP limit 10;""")
df_test.show(5, False)

24/05/27 04:15:32 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/27 04:15:34 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB


+--------+---------------------------------------------+-------+---------------------------------+------------+------+-------------------+---------------+-------------------------------------+------------------------------------+---------------------------------------------------------------------------------+--------------+--------------+---------+-----------+
|PRODUCTO|PRESENTACIÓN                                 |MARCA  |CATEGORÍA                        |CATÁLOGO    |PRECIO|FECHAREGISTRO      |CADENACOMERCIAL|GIRO                                 |NOMBRECOMERCIAL                     |DIRECCIÓN                                                                        |ESTADO        |MUNICIPIO     |LATITUD  |LONGITUD   |
+--------+---------------------------------------------+-------+---------------------------------+------------+------+-------------------+---------------+-------------------------------------+------------------------------------+-------------------------------------------

In [81]:
# #predicción de utiles escolares a partir de estado, municipio y cadenaComecial
# df_utiles = spark.sql("""
#                         Select estado, municipio, cadenaComercial
#                       ,cast(precio as float) from quien 
#                       where categoria = 'MATERIAL ESCOLAR' 
#                       and precio is not null 
#                       and estado is not null
#                       and municipio is not null
#                       and cadenaComercial is not null
#                       """)
# df_utiles.show()

In [82]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
/dev/root        97G   84G   14G  86% /
tmpfs           7.9G     0  7.9G   0% /dev/shm
tmpfs           3.2G  1.2M  3.2G   1% /run
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs           4.0M     0  4.0M   0% /sys/fs/cgroup
efivarfs         56K   24K   27K  48% /sys/firmware/efi/efivars
/dev/sda15      105M  6.1M   99M   6% /boot/efi
tmpfs           1.6G  4.0K  1.6G   1% /run/user/117
tmpfs           1.6G  4.0K  1.6G   1% /run/user/116
tmpfs           1.6G  4.0K  1.6G   1% /run/user/115
tmpfs           1.6G  4.0K  1.6G   1% /run/user/119
tmpfs           1.6G  4.0K  1.6G   1% /run/user/126
tmpfs           1.6G  4.0K  1.6G   1% /run/user/120


In [80]:
spark.stop()